In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import h5py
from sklearn.preprocessing import MinMaxScaler
import warnings
import tqdm
warnings.filterwarnings('ignore')

2023-10-10 16:07:06.039290: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 16:07:08.083694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("../../../data/merge.csv")

In [3]:
df.shape

(1265657, 35)

In [3]:
df.head()

,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
0,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-21 05:55:00,noise,109C.TA_201510210555_NO
1,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-06 14:50:00,noise,109C.TA_201511061450_NO
2,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-07 02:20:00,noise,109C.TA_201511070220_NO
3,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-14 05:15:00,noise,109C.TA_201511140515_NO
4,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-25 18:50:00,noise,109C.TA_201512251850_NO


In [4]:
df = df[df["trace_category"] == "earthquake_local"]

In [5]:
df = df[["trace_start_time","source_latitude","source_longitude","source_magnitude", "trace_name"]]

In [6]:
df["trace_start_time"] = pd.to_datetime(df["trace_start_time"])
df["label"] = np.where(df["source_magnitude"] >= 5, 1, 0)
df["energy"] = np.power(10, 1.44 * df["source_magnitude"] + 5.24)

In [7]:
geo_split = 10
time_split = 30

In [8]:
df["source_latitude"] = df["source_latitude"] // geo_split
df["source_latitude"] = df["source_latitude"] + np.abs(np.min(df["source_latitude"]))

df["source_longitude"] = df["source_longitude"] // geo_split
df["source_longitude"] = df["source_longitude"] + np.abs(np.min(df["source_longitude"]))

In [9]:
# group the data into 30 days intervals
df["trace_start_time"] = df["trace_start_time"].dt.floor('d')
df["trace_start_time"] = df["trace_start_time"] - np.min(df["trace_start_time"])
df["trace_start_time"] = df["trace_start_time"].dt.days // time_split


In [10]:
# group by time and location, put trace_name into a list
df_agg = df.groupby(["trace_start_time","source_latitude","source_longitude"]).agg({"trace_name": lambda x: list(x), "label": np.max, "energy": np.sum}).reset_index()
df_agg

,trace_start_time,source_latitude,source_longitude,trace_name,label,energy
0,0,8.0,5.0,"[BSR.NC_19840430190915_EV, CAL.NC_198404242218...",0,1.889987e+08
1,3,8.0,5.0,[CAL.NC_19840723145533_EV],0,3.808904e+08
2,4,8.0,5.0,[CAL.NC_19840919032005_EV],0,5.733238e+05
3,5,8.0,5.0,"[BSR.NC_19841013082609_EV, CAL.NC_198410061322...",0,2.083121e+08
4,6,8.0,5.0,"[CAL.NC_19841023020631_EV, CAL.NC_198410302048...",0,9.605618e+06
...,...,...,...,...,...,...
6555,422,10.0,1.0,"[PS1A.AV_20181231135051_EV, PS4A.AV_2018123102...",1,3.909421e+13
6556,422,10.0,2.0,"[ACH.AV_20181224082251_EV, CHGN.AT_20181227133...",0,2.152322e+12
6557,422,11.0,1.0,"[G16K.TA_20181230170635_EV, H16K.TA_2018123017...",0,1.592763e+11
6558,422,11.0,2.0,"[CUT.AK_20181227142125_EV, D20K.TA_20181231191...",0,1.663357e+13


In [11]:
def make_ds(df_agg, block_size):
    dfs_train, dfs_val = [], []
    for i in df_agg["source_latitude"].unique():
        for j in df_agg["source_longitude"].unique():
            tmp = df_agg[(df_agg["source_latitude"] == i) & (df_agg["source_longitude"] == j)]
            if not tmp.empty:
                start = max(tmp["trace_start_time"].min() - block_size, 0)
                end = min(tmp["trace_start_time"].max() + block_size, df_agg["trace_start_time"].max())
                tmp = tmp.set_index("trace_start_time").reindex(range(start, end)).fillna(0).rename_axis('trace_start_time')
                tmp["label"] = tmp["label"].shift(-1)
                tmp["source_latitude"] = i
                tmp["source_longitude"] = j
                n = int(0.8 * len(tmp))
                df_train = tmp[:n]
                df_val = tmp[n:]  
                scaler = MinMaxScaler()  
                df_train["energy"] = scaler.fit_transform(df_train["energy"].values.reshape(-1,1))
                df_val["energy"] = scaler.transform(df_val["energy"].values.reshape(-1,1))
                for i in range(block_size):
                    df_train["trace_name_" + str(i)] = df_train["trace_name"].shift(i)
                    df_val["trace_name_" + str(i)] = df_val["trace_name"].shift(i)
                    df_train["energy_" + str(i)] = df_train["energy"].shift(i)
                    df_val["energy_" + str(i)] = df_val["energy"].shift(i)
                dfs_train.append(df_train)
                dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [12]:
block_size = 16
df_train, df_val = make_ds(df_agg, block_size)
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)

In [13]:
df_train

,source_latitude,source_longitude,trace_name,label,energy,trace_name_0,energy_0,trace_name_1,energy_1,trace_name_2,...,trace_name_11,energy_11,trace_name_12,energy_12,trace_name_13,energy_13,trace_name_14,energy_14,trace_name_15,energy_15
trace_start_time,,,,,,,,,,,,,,,,,,,,,
15,8.0,5.0,"[BSR.NC_19850813092346_EV, CAL.NC_198508050539...",0.0,2.235380e-06,"[BSR.NC_19850813092346_EV, CAL.NC_198508050539...",2.235380e-06,"[CAL.NC_19850626185144_EV, CAL.NC_198507171358...",1.304134e-07,"[CAL.NC_19850525104928_EV, CAL.NC_198506071357...",...,[CAL.NC_19840919032005_EV],7.071481e-09,[CAL.NC_19840723145533_EV],4.697972e-06,0,0.000000e+00,0,0.000000e+00,"[BSR.NC_19840430190915_EV, CAL.NC_198404242218...",2.331144e-06
16,8.0,5.0,0,0.0,0.000000e+00,0,0.000000e+00,"[BSR.NC_19850813092346_EV, CAL.NC_198508050539...",2.235380e-06,"[CAL.NC_19850626185144_EV, CAL.NC_198507171358...",...,"[BSR.NC_19841013082609_EV, CAL.NC_198410061322...",2.569359e-06,[CAL.NC_19840919032005_EV],7.071481e-09,[CAL.NC_19840723145533_EV],4.697972e-06,0,0.000000e+00,0,0.000000e+00
17,8.0,5.0,[CAL.NC_19850925234926_EV],0.0,7.839898e-07,[CAL.NC_19850925234926_EV],7.839898e-07,0,0.000000e+00,"[BSR.NC_19850813092346_EV, CAL.NC_198508050539...",...,"[CAL.NC_19841023020631_EV, CAL.NC_198410302048...",1.184775e-07,"[BSR.NC_19841013082609_EV, CAL.NC_198410061322...",2.569359e-06,[CAL.NC_19840919032005_EV],7.071481e-09,[CAL.NC_19840723145533_EV],4.697972e-06,0,0.000000e+00
18,8.0,5.0,"[CAL.NC_19851019074437_EV, CAL.NC_198511061211...",0.0,1.064094e-06,"[CAL.NC_19851019074437_EV, CAL.NC_198511061211...",1.064094e-06,[CAL.NC_19850925234926_EV],7.839898e-07,0,...,"[BSR.NC_19841126151234_EV, BSR.NC_198412011643...",1.004256e-05,"[CAL.NC_19841023020631_EV, CAL.NC_198410302048...",1.184775e-07,"[BSR.NC_19841013082609_EV, CAL.NC_198410061322...",2.569359e-06,[CAL.NC_19840919032005_EV],7.071481e-09,[CAL.NC_19840723145533_EV],4.697972e-06
19,8.0,5.0,"[BSR.NC_19851118111013_EV, BSR.NC_198511220031...",0.0,6.831256e-06,"[BSR.NC_19851118111013_EV, BSR.NC_198511220031...",6.831256e-06,"[CAL.NC_19851019074437_EV, CAL.NC_198511061211...",1.064094e-06,[CAL.NC_19850925234926_EV],...,"[BSR.NC_19841228033300_EV, BSR.NC_198501041959...",7.183874e-06,"[BSR.NC_19841126151234_EV, BSR.NC_198412011643...",1.004256e-05,"[CAL.NC_19841023020631_EV, CAL.NC_198410302048...",1.184775e-07,"[BSR.NC_19841013082609_EV, CAL.NC_198410061322...",2.569359e-06,[CAL.NC_19840919032005_EV],7.071481e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,5.0,10.0,0,0.0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,...,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00
363,5.0,10.0,0,0.0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,...,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00
364,5.0,10.0,0,0.0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,...,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0.000000e+00


In [15]:
class FeatureExtraction(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(16, 3, activation='relu')
        self.pool1 = tf.keras.layers.MaxPooling2D()
        self.conv2 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.pool2 = tf.keras.layers.MaxPooling2D()
        self.conv3 = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.pool3 = tf.keras.layers.MaxPooling2D()
        self.conv4 = tf.keras.layers.Conv2D(128, 3, activation='relu')
        self.pool4 = tf.keras.layers.MaxPooling2D()
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(d_model, activation='relu')

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.conv4(x)
        x = self.pool4(x)
        x = self.flatten(x)
        x = self.d1(x)
        return x

In [16]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.fe = FeatureExtraction(d_model)
        self.pos_encoding = tf.keras.layers.Embedding(100, d_model)

    def call(self, x):
        specs = []
        length = tf.shape(x)[1]
        for i in range(tf.shape(x)[0]):
            spec = self.fe(x[i])
            specs.append(spec)
        x = tf.stack(specs, axis=0)
        x_pos = tf.range(length, dtype=tf.int32)
        x_pos = self.pos_encoding(x_pos)
        x = x + x_pos
        return x

In [17]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [18]:
class CrossAttention(BaseAttention):
    def call(self, x, context):
        attn_output = self.mha(
          query=x,
          key=context,
          value=context)

        x = self.add([x, attn_output])
        x = self.layernorm(x)

        return x

In [19]:
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [20]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model),
        tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [21]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads,
                dff, dropout_rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.emb_x = tf.keras.layers.Embedding(100, d_model)
        self.emb_y = tf.keras.layers.Embedding(100, d_model)
        self.conc = tf.keras.layers.Concatenate(axis=-2)
        self.dense = tf.keras.layers.Dense(4 * d_model)
        self.dense2 = tf.keras.layers.Dense(d_model)

    def call(self, features):
        pos = features
        x = self.emb_x(pos[:,0][:, tf.newaxis])
        y = self.emb_y(pos[:,1][:, tf.newaxis])
        x = self.conc([x,y])
        x = self.dense(x)
        x = self.dense2(x)
        return x  # Shape `(batch_size, seq_len, d_model)`.

In [22]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,
                *,
                d_model,
                num_heads,
                dff,
                dropout_rate=0.1):
        super(DecoderLayer, self).__init__()

        self.causal_self_attention = GlobalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.cross_attention = CrossAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x, context):
        x = self.causal_self_attention(x=x)
        x = self.cross_attention(x=x, context=context)

        x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
        return x

In [23]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff,
                dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(d_model=d_model,)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_layers = [
            DecoderLayer(d_model=d_model, num_heads=num_heads,
                        dff=dff, dropout_rate=dropout_rate)
            for _ in range(num_layers)]

        self.last_attn_scores = None

    def call(self, x, context):
        # `x` is token-IDs shape (batch, target_seq_len)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

        x = self.dropout(x)

        for i in range(self.num_layers):
            x  = self.dec_layers[i](x, context)

        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [24]:
class Transformer(tf.keras.Model):
    def __init__(self, *, num_layers, d_model, num_heads,
                dropout_rate=0.1):
        super().__init__()
        self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=4*d_model,
                            dropout_rate=dropout_rate)

        self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=4*d_model,
                            dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(2)

    def call(self, inputs):
        # To use a Keras model with `.fit` you must pass all your inputs in the
        # first argument.
        x, context  = inputs
        context = self.encoder(context)  # (batch_size, context_len, d_model)
        x = self.decoder(x, context) # (batch_size, target_len, d_model)
        x = x[:, -1, :]
        logits = self.final_layer(x)  # (batch_size, 1, target_vocab_size)

        return logits

In [25]:
def make_ds(df_agg, block_size):
    dfs_train, dfs_val = [], []
    for i in df_agg["source_latitude"].unique():
        for j in df_agg["source_longitude"].unique():
            tmp = df_agg[(df_agg["source_latitude"] == i) & (df_agg["source_longitude"] == j)]
            if not tmp.empty:
                start = max(tmp["time"].min() - block_size, 0)
                end = min(tmp["time"].max() + block_size, df_agg["time"].max())
                tmp = tmp.set_index("time").reindex(range(start, end)).fillna(0).rename_axis('time').reset_index()
                tmp["label"] = tmp["label"].shift(-1)
                tmp["source_latitude"] = i
                tmp["source_longitude"] = j
                n = int(0.8 * len(tmp))
                df_train = tmp[:n]
                df_val = tmp[n:]
                scaler = MinMaxScaler()
                df_train["energy"] = scaler.fit_transform(df_train[["energy"]])
                df_val["energy"] = scaler.transform(df_val[["energy"]])
                for idx in range(1, block_size):
                    df_train["energy" + str(idx)] = df_train["energy"].shift(idx)
                    df_val["energy" + str(idx)] = df_val["energy"].shift(idx)  
                    df_train["trace_name" + str(idx)] = df_train["trace_name"].shift(idx)
                    df_val["trace_name" + str(idx)] = df_val["trace_name"].shift(idx)    
                dfs_train.append(df_train)
                dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [26]:
model = Transformer(num_layers=2, d_model=69, num_heads=3, dropout_rate=0.1)

2023-10-10 02:04:09.563456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 02:04:09.660201: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 02:04:09.660431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 02:04:09.666094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 02:04:09.666274: I tensorflow/compile

In [30]:
def get_spectrogram(name):
    spec = np.load("../../../data/specs/" + name + ".npy")
    return spec

In [34]:
def get_spectrograms(df: pd.DataFrame) -> np.ndarray:
    all_specs = []
    for index, row in df.iterrows():    
        row_specs = []
        for ts in row.values[5:][::2]:
            spec = None
            if type(ts) is list:
                for eq in ts:
                    if spec is None:
                        spec = get_spectrogram(eq)
                    else:
                        spec += get_spectrogram(eq)
            else:
                spec = np.zeros((64, 65, 3))
            row_specs.append(spec)
        row_specs = np.array(row_specs)
        all_specs.append(row_specs)
    return abs(np.array(all_specs))

In [39]:
# training loop for model
EPOCHS = 10
BATCH_SIZE = 32
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

In [40]:
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()

    for idx in range(0, len(df_train), BATCH_SIZE):
        print(idx)
        batch = df_train[idx:idx+BATCH_SIZE]
        x_train = get_spectrograms(batch)
        x_train_pos = batch[["source_latitude", "source_longitude"]].to_numpy()
        y_train = batch["label"].to_numpy().reshape(-1,1)
        y_train = np.concatenate((y_train, 1 - y_train), axis=1)
        with tf.GradientTape() as tape:
            predictions = model([x_train, x_train_pos])
            loss = loss_object(y_train, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)
        train_accuracy(y_train, predictions)

    for idx in range(0, len(df_val), BATCH_SIZE):
        batch = df_val[idx:idx+BATCH_SIZE]
        x_val = get_spectrograms(batch)
        x_val_pos = batch[["source_latitude", "source_longitude"]].to_numpy()
        y_val = batch["label"].to_numpy().reshape(-1,1)
        y_val = np.concatenate((y_val, 1 - y_val), axis=1)
        predictions = model([x_val, x_val_pos])
        loss = loss_object(y_val, predictions)
        val_loss(loss)
        val_accuracy(y_val, predictions)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        val_loss.result(),
                        val_accuracy.result()*100))

    

0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
Epoch 1, Loss: 0.1967088282108307, Accuracy: 94.8233871459961, Val Loss: 0.39201611280441284, Val Accuracy: 95.22183990478516
0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
Epoch 2, Loss: 0.22842839360237122, Accuracy: 94.8233871459961, Val Loss: 0.36006394028663635, Val Accuracy: 95.22183990478516
0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
Epoch 3, Loss: 0.2290467619895935, Accuracy: 94.8233871459961, Val Loss: 0.250

In [37]:
len(df_train), len(df_val)

(1642, 293)